In [1]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-news-extractive-qa/resolve/main/news-extractive-qa.json

In [2]:
import json
import random

In [3]:
with open('news-extractive-qa.json') as fopen:
    data = json.load(fopen)

In [4]:
len(data)

63202

In [5]:
data[0]

['Kuala Terengganu: Kementerian Perdagangan Dalam Negeri, Koperasi dan Kepenggunaan (KPDNKK) Terengganu merampas baju dan seluar jenama tiruan dianggar bernilai RM10,000 dalam serbuan sempena Ramadan di Kampung Tiong, di sini, petang semalam.\n\nDalam serbuan kira-kira 4.30 petang itu, KPDNKK menahan tiga individu termasuk pemilik premis berusia 50-an untuk siasatan mengikut Seksyen 8(2)(b) Akta Perihal Dagangan 2011.\n\nPengarah KPDNKK Terengganu Azlan Abdul Samad berkata, serbuan dilakukan di tiga premis termasuk sebuah stor di kampung sama dalam operasi diketuai tiga pegawai dan sembilan anggota hasil maklumat dan intipan sejak 8 Mei lalu.\n\n"Antara jenama dirampas ialah Adidas, Guess, Gucci, Levi\'s dan Nike dengan bekalan dipercayai diperoleh dari tiga negara jiran.\n\n"Siasatan mendapati pemilik premis berkenaan yang sudah 25 tahun mengaut keuntungan mudah menjalankan perniagaan pakaian tiruan itu dipercayai menjadi pembekal kepada peniaga kecil termasuk di pasar malam di negeri

In [6]:
data_qa = []
for d in data:
    try:
        t = d[0]
        a = d[1]
        j_a = json.loads(d[1])
        data_qa.append(
            {
                'context': t,
                'qa': j_a
            }
        )
    except Exception as e:
        pass
    
len(data_qa)

63013

In [7]:
data_qa[0]

{'context': 'Kuala Terengganu: Kementerian Perdagangan Dalam Negeri, Koperasi dan Kepenggunaan (KPDNKK) Terengganu merampas baju dan seluar jenama tiruan dianggar bernilai RM10,000 dalam serbuan sempena Ramadan di Kampung Tiong, di sini, petang semalam.\n\nDalam serbuan kira-kira 4.30 petang itu, KPDNKK menahan tiga individu termasuk pemilik premis berusia 50-an untuk siasatan mengikut Seksyen 8(2)(b) Akta Perihal Dagangan 2011.\n\nPengarah KPDNKK Terengganu Azlan Abdul Samad berkata, serbuan dilakukan di tiga premis termasuk sebuah stor di kampung sama dalam operasi diketuai tiga pegawai dan sembilan anggota hasil maklumat dan intipan sejak 8 Mei lalu.\n\n"Antara jenama dirampas ialah Adidas, Guess, Gucci, Levi\'s dan Nike dengan bekalan dipercayai diperoleh dari tiga negara jiran.\n\n"Siasatan mendapati pemilik premis berkenaan yang sudah 25 tahun mengaut keuntungan mudah menjalankan perniagaan pakaian tiruan itu dipercayai menjadi pembekal kepada peniaga kecil termasuk di pasar mala

In [8]:
indices = {i for i in range(len(data_qa))}

In [9]:
i = 0
for s in data_qa[i]['qa']['soalan']:
    print(s)

{'soalan': 'Di mana serbuan tersebut dilakukan?', 'substring': 'Kuala Terengganu'}
{'soalan': 'Berapa nilai baju dan seluar jenama tiruan yang dirampas?', 'substring': 'RM10,000'}
{'soalan': 'Berapa individu yang ditahan dalam serbuan tersebut?', 'substring': 'tiga individu'}
{'soalan': 'Apakah jenama-jenama yang dirampas?', 'substring': "Adidas, Guess, Gucci, Levi's dan Nike"}
{'soalan': 'Berapa lama pemilik premis tersebut menjalankan perniagaan pakaian tiruan?', 'substring': '25 tahun'}


In [10]:
from tqdm import tqdm

pairs = []
for i in tqdm(range(len(data_qa))):
    query = data_qa[i]['context']
    pos = []
    for s in data_qa[i]['qa']['soalan']:
        try:
            pos.append(s['soalan'])
        except:
            pass
    
    neg_i = random.choice(list(indices - {i}))
    neg = []
    for s in data_qa[neg_i]['qa']['soalan']:
        try:
            neg.append(s['soalan'])
        except:
            pass
        
    pairs.append({
        'query': query,
        'pos': pos,
        'neg': neg
    })
    
    for s in data_qa[i]['qa']['soalan']:
        try:
            pos = [query]
            neg_i = random.choice(list(indices - {i}))
            neg = [data_qa[neg_i]['context']]

            pairs.append({
                'query': s['soalan'],
                'pos': pos,
                'neg': neg
            })
        except:
            pass

100%|████████████████████████████████████| 63013/63013 [03:27<00:00, 303.45it/s]


In [11]:
with open('news-qa-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')

In [12]:
len(pairs)

378082